<a href="https://colab.research.google.com/github/mariskaiz/TugasML/blob/main/ML3_Kel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tensorflow pandas numpy scikit-learn


In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re





In [5]:
data = pd.read_csv('/content/Sentiment.csv')
# menjaga kolom yang diperlukan
data = data[['text','sentiment']]

In [6]:
#prprocessing teks
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

#mencetak ukuran data
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

#pembersihan teks dari retweet
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

#tokenisasi dan padding untuk memastikan panjang yang sama
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [8]:
#menetapkan dimensi embedding dan ukuran LSTM
embed_dim = 128
lstm_out = 196

#pembuatan model
model = Sequential()
#penambahan layer embedding, spatialDropout1D, LSTM, dense
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spati  (None, 28, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
#pengubahan label menjadi one-hot encoding
Y = pd.get_dummies(data['sentiment']).values

#pembagian data train dan data test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [10]:
#ukuran batch
batch_size = 32

#melatih model
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
225/225 - 39s - loss: 0.4386 - accuracy: 0.8155 - 39s/epoch - 172ms/step
Epoch 2/7
225/225 - 34s - loss: 0.3163 - accuracy: 0.8655 - 34s/epoch - 153ms/step
Epoch 3/7
225/225 - 35s - loss: 0.2753 - accuracy: 0.8859 - 35s/epoch - 154ms/step
Epoch 4/7
225/225 - 32s - loss: 0.2454 - accuracy: 0.8996 - 32s/epoch - 143ms/step
Epoch 5/7
225/225 - 34s - loss: 0.2169 - accuracy: 0.9124 - 34s/epoch - 152ms/step
Epoch 6/7
225/225 - 32s - loss: 0.1988 - accuracy: 0.9182 - 32s/epoch - 144ms/step
Epoch 7/7
225/225 - 33s - loss: 0.1719 - accuracy: 0.9307 - 33s/epoch - 148ms/step


In [11]:
#ukuran validasi
validation_size = 1500

#membagi data uji dan validasi
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

#evaluasi model
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

64/64 - 2s - loss: 0.4920 - accuracy: 0.8422 - 2s/epoch - 24ms/step
score: 0.49
acc: 0.84


In [12]:
#inisiasi variabel penghitung
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0

#prediksi dan evaluasi
for x in range(len(X_validate)):

    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]

    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

#menghitung jumlah total sampel
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

#menampilkan akurasi
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 267ms/epoch - 267ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 4

In [13]:
#definisi tweet untuk prediksi
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vektorisasi tweet
twt = tokenizer.texts_to_sequences(twt)
#padding tweet untuk menyamakan shape
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)

#memprediksi sentimen
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
#menentukan dan menampilan sentimen
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  204  625    7  152    6   56 1039   56   47    7  152]]
1/1 - 0s - 30ms/epoch - 30ms/step
negative
